## **챗 모델과 Message를 사용해 간단한 LLM 애플리케이션 구축하기**

In [ ]:
# os.environ

## 언어 모델 사용하기

LangChain은 다양한 언어 모델을 지원하며, 이들을 서로 교체하여 사용할 수 있습니다.

In [ ]:
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

ChatModels은 LangChain Runnables의 인스턴스로, 표준화된 인터페이스를 통해 상호작용할 수 있습니다. 모델을 간단히 호출하려면 `.invoke` 메서드에 Messages 목록을 전달하면 됩니다.

## 메시지 (Messages)

메시지(Messages) 는 LangChain에서 모델이 사용하는 컨텍스트의 기본 단위입니다.
이들은 모델의 입력(input) 과 출력(output) 을 나타내며, LLM과의 상호작용에서 대화의 상태(state)를 표현하는 데 필요한 콘텐츠(content) 와 메타데이터(metadata) 를 함께 포함합니다.

메시지는 다음 요소들로 구성됩니다:

- Role (역할)
→ 메시지의 유형을 식별합니다. 예: system, user, assistant 등  
- Content (내용)
→ 메시지의 실제 내용으로, 텍스트뿐만 아니라 이미지, 오디오, 문서 등 다양한 형식을 포함할 수 있습니다.  
- Metadata (메타데이터)
→ 선택적(optional) 필드로, 응답 정보(response info), 메시지 ID, 토큰 사용량(token usage) 등 부가 정보를 담습니다.

In [ ]:
# 메시지 목록을 생성
    # 시스템 메시지: 모델에게 수행할 작업이나 역할을 지시합니다.
    # 사용자 메시지: 사용자가 모델에 보낼 실제 입력 내용입니다.

### 텍스트 프롬프트 (Text prompts)

**텍스트 프롬프트(Text prompts)** 는 단순한 **문자열(string)** 형태로 제공됩니다.  
대화 기록(conversation history)을 유지할 필요가 없는 **단순한 생성 작업**(예: 요약, 문장 생성, 번역 등)에 적합합니다.

### 딕셔너리 형식 (Dictionary format)

OpenAI의 **Chat Completions 포맷**을 사용해 메시지를 **딕셔너리 형태로 직접 지정**할 수도 있습니다.

In [ ]:
# OpenAI 형식

--------------
## 메시지 유형 (Message types)

* **System message (시스템 메시지)**
  → 모델이 어떻게 행동해야 하는지 지시하고, 상호작용의 **맥락(context)** 을 제공합니다.  
* **Human message (사용자 메시지)**
  → 사용자의 입력을 나타내며, 모델과의 **대화(interaction)** 를 구성합니다.  
* **AI message (AI 메시지)**
  → 모델이 생성한 응답으로, **텍스트 내용(text content)** 뿐 아니라
  **도구 호출(tool calls)** 및 **메타데이터(metadata)** 를 포함할 수 있습니다.  
* **Tool message (도구 메시지)**
  → 모델이 호출한 **도구의 실행 결과(outputs)** 를 나타냅니다.

### AI 메시지 (AI Message)

**AIMessage** 는 **모델 호출(model invocation)** 의 **출력 결과**를 나타냅니다.
이 메시지에는 다음과 같은 요소들을 포함할 수 있습니다:

* **멀티모달 데이터 (Multimodal data)** — 텍스트뿐만 아니라 이미지, 오디오 등의 출력
* **도구 호출 (Tool calls)** — 모델이 외부 도구를 실행한 기록
* **제공자별 메타데이터 (Provider-specific metadata)** — 모델 제공자(OpenAI, Anthropic 등)에 따라 추가로 제공되는 정보

이러한 메타데이터는 나중에 접근하거나 분석할 수 있습니다.

In [ ]:
# AI 메시지를 수동으로 생성 (예: 대화 기록에 추가하기 위해)
# 대화 기록에 추가

### 토큰 사용량 (Token usage)

**AIMessage** 객체는 `usage_metadata` 필드에
**토큰 사용량(token counts)** 및 기타 **사용 관련 메타데이터(metadata)** 를 저장할 수 있습니다.

### batch interface 이용 여러개의 메시지 일괄 처리

In [ ]:
# 메시지 목록을 생성
# `model.batch()`을 사용하여 여러 개의 메시지를 한 번에 처리
# LangChain은 각 입력을 독립된 invoke() 호출처럼 처리

In [ ]:
# 결과 출력

### stream inferface 를 이용한 출력 

In [ ]:
# 스트리밍 중에 받아온 chunk 들을 저장할 리스트
# 전체 메시지를 누적할 변수
# model.stream() 은 토큰(혹은 chunk)을 스트리밍 방식으로 하나씩 생성
    # full_message 에 chunk 를 누적
    # 첫 chunk 일 경우 full_message 가 None 이므로 그대로 저장
    # 이후부터는 기존 full_message 에 chunk 를 더해(concat) 전체 메시지를 구성

### 도구 메시지 (Tool Message)

도구 호출(tool calling)을 지원하는 모델의 경우,
**AI 메시지(AIMessage)** 안에 **도구 호출 정보(tool calls)** 가 포함될 수 있습니다.

**Tool Message(도구 메시지)** 는
**단일 도구 실행의 결과(result)** 를 모델에게 다시 전달하기 위해 사용됩니다.

또한, 도구는 직접 **`ToolMessage` 객체**를 생성하여
실행 결과를 LangChain 에이전트나 모델로 반환할 수 있습니다.

In [ ]:
# 모델이 도구 호출을 수행한 후
# 도구 실행 후 결과 메시지 생성
# 대화 이어가기